In [185]:
# Dependencies and Setup
%matplotlib inline
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
from scipy.stats import sem

# Hide warning messages in notebook
import warnings
warnings.filterwarnings('ignore')

# File to Load (Remember to Change These)
crime_data_to_load = "data/Boston Crime Data.csv"
zip_data_to_load = "data/Boston_ZipCodes.csv"
dist_data_to_load = "data/districts.csv"
census_data_to_load = "data/Census Race Stats.csv"

# Read Data 
crime_data = pd.read_csv(crime_data_to_load)
# Prevent truncation of zero in Zipcode 
zip_data = pd.io.parsers.read_csv(zip_data_to_load, dtype={'Zip Code ': 'str'}).set_index('Zip Code ')
dist_data = pd.read_csv(dist_data_to_load)
cen_data = pd.io.parsers.read_csv(census_data_to_load, dtype={'Zip Code': 'str'})




# Combine the data into a single dataset
# data_combine = pd.merge(mouse_data,clinical_data, how='left', on=["Mouse ID", "Mouse ID"])
# Display the data table for preview
cen_data.head(2)


,Unnamed: 0,Zip Code,District,White Pop,Afr Am Pop,Asian Pop,Native Am Pacific Pop,Other Pop,Multi Race Pop
0,0,02108,Boston,3563.0,223.0,204.0,0.0,0.0,165.0
1,1,02109,Boston,3419.0,157.0,343.0,0.0,0.0,68.0


#                                    Transformation - Crime Data 

In [186]:
crime_data.columns

Index(['INCIDENT_NUMBER', 'OFFENSE_CODE', 'OFFENSE_CODE_GROUP',
       'OFFENSE_DESCRIPTION', 'DISTRICT', 'REPORTING_AREA', 'SHOOTING',
       'OCCURRED_ON_DATE', 'YEAR', 'MONTH', 'DAY_OF_WEEK', 'HOUR', 'UCR_PART',
       'STREET', 'Lat', 'Long', 'Location'],
      dtype='object')

In [187]:
# kept occurred_on_date because the stripped out date didnt contain "day", easier to parse out the date later into the units needed 
Crime_data_trunc = crime_data[['OFFENSE_CODE','OFFENSE_CODE_GROUP',
        'DISTRICT', 'REPORTING_AREA', 'SHOOTING',
        'OCCURRED_ON_DATE']]

In [188]:
Crime_data_trunc.set_index('DISTRICT')

,OFFENSE_CODE,OFFENSE_CODE_GROUP,REPORTING_AREA,SHOOTING,OCCURRED_ON_DATE
DISTRICT,,,,,
D14,619,Larceny,808,NaN,9/2/2018 13:00
C11,1402,Vandalism,347,NaN,8/21/2018 0:00
D4,3410,Towed,151,NaN,9/3/2018 19:27
D4,3114,Investigate Property,272,NaN,9/3/2018 21:16
B3,3114,Investigate Property,421,NaN,9/3/2018 21:05
C11,3820,Motor Vehicle Accident Response,398,NaN,9/3/2018 21:09
B2,724,Auto Theft,330,NaN,9/3/2018 21:25
B2,3301,Verbal Disputes,584,NaN,9/3/2018 20:39
C6,301,Robbery,177,NaN,9/3/2018 20:48


In [1]:
# data_combine = pd.merge(Crime_data_trunc,clinical_data, how='left', on=["District", "District"])

#                               Transformation - District 

In [189]:
dist_data   

,Unnamed: 0,District Codes,District Names
0,0,District A-7,"69 Paris Street, East Boston, MA 02128"
1,1,District B-2,"2400 Washington Street, Roxbury, MA 02119"
2,2,District B-3,"1165 Blue Hill Avenue, Mattapan, MA 02124"
3,3,District C-6,"101 West Broadway, South Boston, MA 02127"
4,4,District C-11,"40 Gibson Street, Dorchester, MA 02122"
5,5,District D-4,"650 Harrison Avenue, Boston, MA 02116"
6,6,District D-14,"301 Washington Street, Brighton, MA 02135"
7,7,District E-5,"1708 Centre Street, West Roxbury, MA 02132"
8,8,District E-13,"3347 Washington Street, Jamaica Plain, MA 02130"
9,9,District E-18,"1249 Hyde Park Avenue, Hyde Park, MA 02136"


In [190]:
dist_data_clean = pd.DataFrame()
dist_data_clean['District_Codes'] =(dist_data['District Codes'].str.split(' ').str[1])
dist_data_clean['District_Codes2']=dist_data_clean['District_Codes'].str.split('-').str[0]+dist_data_clean['District_Codes'].str.split('-').str[1]
dist_data_clean['District_Address'] =(dist_data['District Names'].str.split(',').str[0])
dist_data_clean['District'] =(dist_data['District Names'].str.split(',').str[1])   
# Clean up Dual Precinct for Boston/Charlestown
# dist_data_clean.replace('40 New Sudbury Street Boston', '40 New Sudbury Street')
dist_data_clean.at[10, 'District_Address'] = '40 New Sudbury Street'
dist_data_clean.at[11, 'District_Address'] = '40 New Sudbury Street'
dist_data_clean.at[10, 'District'] = 'Charlestown'
dist_data_clean.at[11, 'District'] = 'Boston'
dist_data_clean.at[11, 'District_Codes'] = 'A15'
dist_data_clean.at[11, 'District_Codes2'] = 'A15'
dist_data_clean.drop(['District_Codes'], axis=1,inplace=True)
dist_data_clean.rename(columns={'District_Codes2':'District_Codes'},inplace=True)
dist_data_clean

,District_Codes,District_Address,District
0,A7,69 Paris Street,East Boston
1,B2,2400 Washington Street,Roxbury
2,B3,1165 Blue Hill Avenue,Mattapan
3,C6,101 West Broadway,South Boston
4,C11,40 Gibson Street,Dorchester
5,D4,650 Harrison Avenue,Boston
6,D14,301 Washington Street,Brighton
7,E5,1708 Centre Street,West Roxbury
8,E13,3347 Washington Street,Jamaica Plain
9,E18,1249 Hyde Park Avenue,Hyde Park


#                               Transformation - Census Data

In [191]:
cen_data.dropna(inplace=True)
cen_data.drop(['Unnamed: 0', 'Zip Code'], axis=1, inplace=True)
cen_data.head(5)

,District,White Pop,Afr Am Pop,Asian Pop,Native Am Pacific Pop,Other Pop,Multi Race Pop
0,Boston,3563.0,223.0,204.0,0.0,0.0,165.0
1,Boston,3419.0,157.0,343.0,0.0,0.0,68.0
2,Boston,1896.0,92.0,260.0,0.0,0.0,22.0
3,Boston,3250.0,156.0,3673.0,0.0,0.0,301.0
5,Boston,7022.0,52.0,161.0,0.0,0.0,112.0


In [ ]:
# # print(type(dataFF))
# data_frames = [dataFF,dataBB ]

# #merge as needed
# Final_Data2 = reduce(lambda  left,right: pd.merge(left,right,on=['Date', 'Tableau Geo', 'Tableau LOB','TM1 Geography','Account', 'Scenario' ],
#                                             how='outer'), data_frames).fillna(0)

,index,District,White Pop,Afr Am Pop,Asian Pop,Native Am Pacific Pop,Other Pop,Multi Race Pop
0,0,Boston,3563.0,223.0,204.0,0.0,0.0,165.0
1,1,Boston,3419.0,157.0,343.0,0.0,0.0,68.0
2,2,Boston,1896.0,92.0,260.0,0.0,0.0,22.0
3,3,Boston,3250.0,156.0,3673.0,0.0,0.0,301.0
4,5,Boston,7022.0,52.0,161.0,0.0,0.0,112.0
5,6,Boston,10504.0,613.0,1403.0,0.0,0.0,264.0
6,7,Boston,19506.0,2421.0,4531.0,32.0,32.0,948.0
7,8,Boston,17684.0,1297.0,3310.0,0.0,0.0,605.0
8,10,Roxbury,14830.0,4467.0,4363.0,0.0,0.0,1267.0
9,11,Roxbury,4257.0,16232.0,866.0,7.0,7.0,1542.0


# Create CSV for Database creation

In [150]:
zip_data.to_csv('Database_Files/ZipCode_data.csv')
dist_data_clean.to_csv('Database_Files/Police_Districts.csv')
cen_data.to_csv('Database_Files/Census.csv')
crime_data.to_csv('Database_Files/Boston_Crime.csv')


In [152]:
crime_data

,INCIDENT_NUMBER,OFFENSE_CODE,OFFENSE_CODE_GROUP,OFFENSE_DESCRIPTION,DISTRICT,REPORTING_AREA,SHOOTING,OCCURRED_ON_DATE,YEAR,MONTH,DAY_OF_WEEK,HOUR,UCR_PART,STREET,Lat,Long,Location
0,I182070945,619,Larceny,LARCENY ALL OTHERS,D14,808,NaN,9/2/2018 13:00,2018,9,Sunday,13,Part One,LINCOLN ST,42.357791,-71.139371,"(42.35779134, -71.13937053)"
1,I182070943,1402,Vandalism,VANDALISM,C11,347,NaN,8/21/2018 0:00,2018,8,Tuesday,0,Part Two,HECLA ST,42.306821,-71.060300,"(42.30682138, -71.06030035)"
2,I182070941,3410,Towed,TOWED MOTOR VEHICLE,D4,151,NaN,9/3/2018 19:27,2018,9,Monday,19,Part Three,CAZENOVE ST,42.346589,-71.072429,"(42.34658879, -71.07242943)"
3,I182070940,3114,Investigate Property,INVESTIGATE PROPERTY,D4,272,NaN,9/3/2018 21:16,2018,9,Monday,21,Part Three,NEWCOMB ST,42.334182,-71.078664,"(42.33418175, -71.07866441)"
4,I182070938,3114,Investigate Property,INVESTIGATE PROPERTY,B3,421,NaN,9/3/2018 21:05,2018,9,Monday,21,Part Three,DELHI ST,42.275365,-71.090361,"(42.27536542, -71.09036101)"
5,I182070936,3820,Motor Vehicle Accident Response,M/V ACCIDENT INVOLVING PEDESTRIAN - INJURY,C11,398,NaN,9/3/2018 21:09,2018,9,Monday,21,Part Three,TALBOT AVE,42.290196,-71.071590,"(42.29019621, -71.07159012)"
6,I182070933,724,Auto Theft,AUTO THEFT,B2,330,NaN,9/3/2018 21:25,2018,9,Monday,21,Part One,NORMANDY ST,42.306072,-71.082733,"(42.30607218, -71.08273260)"
7,I182070932,3301,Verbal Disputes,VERBAL DISPUTE,B2,584,NaN,9/3/2018 20:39,2018,9,Monday,20,Part Three,LAWN ST,42.327016,-71.105551,"(42.32701648, -71.10555088)"
8,I182070931,301,Robbery,ROBBERY - STREET,C6,177,NaN,9/3/2018 20:48,2018,9,Monday,20,Part One,MASSACHUSETTS AVE,42.331521,-71.070853,"(42.33152148, -71.07085307)"
9,I182070929,3301,Verbal Disputes,VERBAL DISPUTE,C11,364,NaN,9/3/2018 20:38,2018,9,Monday,20,Part Three,LESLIE ST,42.295147,-71.058608,"(42.29514664, -71.05860832)"


In [100]:
cen_data.head(2) 


,Unnamed: 0,Zip Code,District,White Pop,Afr Am Pop,Asian Pop,Native Am Pacific Pop,Other Pop,Multi Race Pop
0,0,02108,Boston,3563.0,223.0,204.0,0.0,0.0,165.0
1,1,02109,Boston,3419.0,157.0,343.0,0.0,0.0,68.0


In [115]:
dist_data_clean.head(2) 

,District_Codes,District_Address,District
0,A-7,69 Paris Street,East Boston
1,B-2,2400 Washington Street,Roxbury


In [154]:
zip_data.head

<bound method NDFrame.head of                 District
Zip Code                
02108             Boston
02109             Boston
02110             Boston
02111             Boston
02112             Boston
02113             Boston
02114             Boston
02115             Boston
02116             Boston
02117             Boston
02118            Roxbury
02119            Roxbury
02120            Roxbury
02121         Dorchester
02122         Dorchester
02123             Boston
02124         Dorchester
02125         Dorchester
02126           Mattapan
02127       South Boston
02128        East Boston
02129        Charlestown
02130      Jamaica Plain
02131        Roslindale 
02132       West Roxbury
02133             Boston
02134            Allston
02135           Brighton
02136          Hyde Park
02137          Hyde Park>